In [224]:
!pip install -q yfinance

In [225]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# Lendo o arquivo de excel com a base de dados.
* Nesse caso está lendo a aba "cv" da coluna A até H. ✅

In [226]:
base_dados = ('/content/holding.xlsm')
df = pd.read_excel(base_dados, sheet_name='cv', usecols="A:H")
df

,DATA,MÊS,ANO,C/V,ATIVO,QUANTIDADE,PREÇO,VALOR TOTAL
0,2020-02-14,FEB,2020,COMPRA,BIDI4,100,16.190000,1619.000000
1,2020-02-14,FEB,2020,COMPRA,BOVA11,10,110.190000,1101.900000
2,2020-02-14,FEB,2020,COMPRA,COGN3,100,11.780000,1178.000000
3,2020-02-14,FEB,2020,COMPRA,ECOR3,100,17.120000,1712.000000
4,2020-02-17,FEB,2020,COMPRA,MRFG3,100,12.340000,1234.000000
...,...,...,...,...,...,...,...,...
1071,2023-05-31,MAY,2023,COMPRA,FLRY3,100,15.620000,1562.000000
1072,2023-06-14,JUN,2023,BONIFICAÇÃO,FLRY3,91,6.524294,593.710712
1073,2023-06-16,JUN,2023,COMPRA,EGIE3,53,45.190000,2395.070000
1074,2023-06-16,JUN,2023,COMPRA,TAEE3,100,12.620000,1262.000000


* Calculando a soma de ATIVOS na coluna QUANTIDADE ✅
* Filtrando os ativos com quantidade > 1 para remoção de alguns ativos indesejados.

In [227]:
df_ativos = df.groupby('ATIVO')['QUANTIDADE'].sum()
df_ativos = df_ativos[df_ativos >1]
df_ativos


ATIVO
ABCB4         900
BBAS3        1457
BBDC3         800
BBDC4         900
BBSE3        1900
DOLAR         100
EGIE3         700
FLRY3        1915
IRBR1          28
ITSA4        2500
JHSF3       10694
MXRF12        176
POUPANÇA       44
PSSA3         612
SANB3        1100
SAPR4        4478
TAEE3        1600
TRPL4        2265
Name: QUANTIDADE, dtype: int64

* Criando uma lista com os ativos indesejados e remove-los com a variável df_ativos_filtrados

In [228]:
ativos_remover = ['POUPANÇA', 'MXRF12', 'IRBR1', 'DOLAR']
df_ativos = df_total_ativos[~df_total_ativos.index.isin(ativos_remover)]
df_ativos

ATIVO
ABCB4      900
BBAS3     1457
BBDC3      800
BBDC4      900
BBSE3     1900
EGIE3      700
FLRY3     1915
ITSA4     2500
JHSF3    10694
PSSA3      612
SANB3     1100
SAPR4     4478
TAEE3     1600
TRPL4     2265
Name: QUANTIDADE, dtype: int64

* Convertendo a Series para DataFrame

In [229]:
df_ativos = df_ativos.reset_index()
df_ativos

,ATIVO,QUANTIDADE
0,ABCB4,900
1,BBAS3,1457
2,BBDC3,800
3,BBDC4,900
4,BBSE3,1900
5,EGIE3,700
6,FLRY3,1915
7,ITSA4,2500
8,JHSF3,10694
9,PSSA3,612


# Baixar os dados históricos de fechamento das ações
  - Obtendo e tratando a lista de tickers
  - fazendo o download da cotação de todos os ativos na lista

In [230]:
tickers = [ativo + '.SA' for ativo in df_ativos ['ATIVO']]
tickers

['ABCB4.SA',
 'BBAS3.SA',
 'BBDC3.SA',
 'BBDC4.SA',
 'BBSE3.SA',
 'EGIE3.SA',
 'FLRY3.SA',
 'ITSA4.SA',
 'JHSF3.SA',
 'PSSA3.SA',
 'SANB3.SA',
 'SAPR4.SA',
 'TAEE3.SA',
 'TRPL4.SA']

In [231]:
cotacao_atual = yf.download(tickers, period="1d")["Adj Close"]


[*********************100%***********************]  14 of 14 completed


* Ajustando o dataframe df_cotacao com colunas "ACOES" e "COTACAO"

In [232]:
df_cotacao = cotacao_atual.reset_index()  # Resetando o índice para transformar a data em uma coluna
df_cotacao = cotacao_atual_reset.melt(id_vars='Date', var_name='ACOES', value_name='COTACÃO')
df_cotacao = df_cotacao.drop(columns='Date')
df_cotacao

,ACOES,COTACÃO
0,ABCB4.SA,17.690001
1,BBAS3.SA,49.490002
2,BBDC3.SA,14.540000
3,BBDC4.SA,16.950001
4,BBSE3.SA,30.450001
5,EGIE3.SA,45.200001
6,FLRY3.SA,15.650000
7,ITSA4.SA,9.650000
8,JHSF3.SA,4.950000
9,PSSA3.SA,27.280001


# Mesclando df_ativos com df_cotacao
- df_cotacao
- df_ativos

In [233]:
# Fazendo o merged entre df_ativos e df_cotacao e removendo a coluna 'ACOES'
df_merged = df_ativos.merge(df_cotacao, left_index=True, right_index=True)
df_merged = df_merged.drop(columns='ACOES')

# Adicionando a coluna 'PATRIMONIO' e mutiplicando QUANTIDADE * COTACAO e formatando as casas decimais.
df_merged['PATRIMÔNIO'] = df_merged['COTACÃO'] * df_merged['QUANTIDADE']
df_merged['PATRIMÔNIO'] = df_merged['PATRIMÔNIO']
# .apply(lambda x: 'R$ {:,.2f}'.format(x).replace('.', '#').replace(',', '.').replace('#', ','))
display(df_merged)

# Calculando a soma total dos valores de cada ação

soma_total = df_merged['PATRIMÔNIO'].sum()
print('Patriônio total R$ {:.2f}'.format(soma_total))

,ATIVO,QUANTIDADE,COTACÃO,PATRIMÔNIO
0,ABCB4,900,17.690001,15921.000481
1,BBAS3,1457,49.490002,72106.932446
2,BBDC3,800,14.540000,11631.999969
3,BBDC4,900,16.950001,15255.000687
4,BBSE3,1900,30.450001,57855.001450
5,EGIE3,700,45.200001,31640.000534
6,FLRY3,1915,15.650000,29969.749269
7,ITSA4,2500,9.650000,24124.999046
8,JHSF3,10694,4.950000,52935.297960
9,PSSA3,612,27.280001,16695.360420


Patriônio total R$ 439291.80


In [234]:
# # Calculando a porcentagem de cada ativo com base no valor total
# valor_carteira['Porcentagem'] = ((valor_carteira['Valor_Carteira'].str.replace('R$', '').str.replace('.', '').str.replace(',', '.').astype(float) / soma_total) * 100).round(2)

df_merged['POSIÇÃO %'] = (df_merged['PATRIMÔNIO'] / soma_total) * 100
df_merged

,ATIVO,QUANTIDADE,COTACÃO,PATRIMÔNIO,POSIÇÃO %
0,ABCB4,900,17.690001,15921.000481,3.624243
1,BBAS3,1457,49.490002,72106.932446,16.414359
2,BBDC3,800,14.540000,11631.999969,2.647898
3,BBDC4,900,16.950001,15255.000687,3.472635
4,BBSE3,1900,30.450001,57855.001450,13.170062
5,EGIE3,700,45.200001,31640.000534,7.202502
6,FLRY3,1915,15.650000,29969.749269,6.822287
7,ITSA4,2500,9.650000,24124.999046,5.491794
8,JHSF3,10694,4.950000,52935.297960,12.050145
9,PSSA3,612,27.280001,16695.360420,3.800517


In [235]:
valores_meta = {
    'TRPL4': 13,
    'TAEE3': 5,
    'EGIE3': 7,
    'SANB3': 4,
    'ABCB4': 4,
    'FLRY3': 7,
    'PSSA3': 4,
    'ITSA4': 6,
    'SAPR4': 4,
    'JHSF3': 11,
    'BBDC3': 2,
    'BBSE3': 12,
    'BBDC4': 5,
    'BBAS3': 16
}

df_merged['OBJETIVO %'] = df_merged['ATIVO'].map(valores_meta)
df_merged


,ATIVO,QUANTIDADE,COTACÃO,PATRIMÔNIO,POSIÇÃO %,OBJETIVO %
0,ABCB4,900,17.690001,15921.000481,3.624243,4
1,BBAS3,1457,49.490002,72106.932446,16.414359,16
2,BBDC3,800,14.540000,11631.999969,2.647898,2
3,BBDC4,900,16.950001,15255.000687,3.472635,5
4,BBSE3,1900,30.450001,57855.001450,13.170062,12
5,EGIE3,700,45.200001,31640.000534,7.202502,7
6,FLRY3,1915,15.650000,29969.749269,6.822287,7
7,ITSA4,2500,9.650000,24124.999046,5.491794,6
8,JHSF3,10694,4.950000,52935.297960,12.050145,11
9,PSSA3,612,27.280001,16695.360420,3.800517,4


In [239]:
df_merged['COMPRAR?'] = (df_merged['POSIÇÃO %'] - df_merged['OBJETIVO %'])
df_merged = df_merged.sort_values(by='COMPRAR?')
df_merged

,ATIVO,QUANTIDADE,COTACÃO,PATRIMÔNIO,POSIÇÃO %,OBJETIVO %,COMPRAR?
3,BBDC4,900,16.950001,15255.000687,3.472635,5,-1.527365
7,ITSA4,2500,9.650000,24124.999046,5.491794,6,-0.508206
10,SANB3,1100,14.070000,15476.999664,3.523171,4,-0.476829
12,TAEE3,1600,12.460000,19936.000061,4.538214,5,-0.461786
0,ABCB4,900,17.690001,15921.000481,3.624243,4,-0.375757
9,PSSA3,612,27.280001,16695.360420,3.800517,4,-0.199483
6,FLRY3,1915,15.650000,29969.749269,6.822287,7,-0.177713
13,TRPL4,2265,24.900000,56398.499136,12.838505,13,-0.161495
5,EGIE3,700,45.200001,31640.000534,7.202502,7,0.202502
11,SAPR4,4478,4.320000,19344.960769,4.403670,4,0.403670


In [237]:
soma_objetivo = df_merged['OBJETIVO %'].sum()
soma_objetivo

100